# データ取り込み

このノートブックはETL処理における **E(Extract:抽出)** に該当します。Pythonを用いてインターネットからCSVファイルを取得します。

こちらは、[Azure Databricks ジョブを使用して最初のワークフローを作成する](https://learn.microsoft.com/ja-jp/azure/databricks/jobs/jobs-quickstart)のサンプルノートブックをベースとしています。

まず、このノートブックを[ノートブック用のサーバレスコンピューティング](https://learn.microsoft.com/ja-jp/azure/databricks/compute/serverless/notebooks)で実行し、その後で[ジョブ用のサーバレスコンピューティング](https://learn.microsoft.com/ja-jp/azure/databricks/jobs/run-serverless-jobs)を用いてジョブとして実行します。

ノートブック右上の**接続**から**サーバレス**を選択してください。

![](https://sajpstorage.blob.core.windows.net/yayoi/202412_handson/serverless_notebook.png)

## Unity Catalog

Databricksの資産(テーブルやファイルなど)はすべて[Unity Catalog](https://learn.microsoft.com/ja-jp/azure/databricks/data-governance/unity-catalog/)によって管理されます。データエンジニアリングを行う際には、対象のデータがUnity Catalogのどこに存在しているのかを理解することはとても重要です。

Unity Catalogを用いることで、以下の機能でDatabricks上の資産に対するガバナンスを強化することができます。

- **データアクセス制御**
  - 一度定義するだけで、全てのワークスペース、全ての言語、全てのユースケースに対してセキュリティを適用
- **監査機能**
  - ガバナンスユースケースに対応するために、全てのクエリーに対するきめ細かい監査を実現
- **データリネージ**
  - テーブル、カラムに対するデータリネージの自動収集
- **データ探索**
  - お使いのレイクハウスにおいてデータ利用者が信頼できるデータを検索するためのインタフェースを提供
- **属性ベースのアクセス制御**
  - 行列レベルのネイティブなセキュリティ、タグによるポリシー適用

### 3レベルの名前空間

Unity Catalogはデータを整理するために3レベルの名前空間、カタログ、スキーマ(データベースとも呼ばれます)、テーブルとビューを提供します。テーブルを参照するには以下の文法を使用します。

>`<catalog>.<schema>.<table>`

![3_level_namespace.png](./img/3_level_namespace.png "3_level_namespace.png")

ファイルは[ボリューム](https://learn.microsoft.com/ja-jp/azure/databricks/volumes/)という場所に格納されます。ボリュームはテーブルと同様`カタログ.スキーマ(データベース).ボリューム`という三階層で管理されます。また、ボリュームに格納されているファイルには[パス](https://learn.microsoft.com/ja-jp/azure/databricks/data-governance/unity-catalog/paths)を指定してアクセスすることができます。データエンジニアリングにおいては、ファイルの取り扱いは不可欠ですので、パスの考え方に慣れるようにしましょう。

`<catalog_name>.<schema_name>.<volume_name>`で管理されているボリュームには、PythonやSQLから`/Volumes/<catalog_name>/<schema_name>/<volume_name>/<path_to_file>`というパスでアクセスすることができます。

まず、今回のハンズオンで使用するカタログ、スキーマ、ボリュームを設定します。

In [0]:
%run ./config

## カタログの指定

使用するカタログやスキーマ(データベース)にアクセスするには明示的に指定する必要があります。`USE CATALOG`文を用いて、上で表示される作成済みのカタログを使用します。

In [0]:
# 使用するカタログを指定
spark.sql(f"USE CATALOG {CATALOG_NAME}")

## スキーマ(データベース)の作成

皆様のユーザー名(メールアドレス)をベースとしたスキーマを作成します。

In [0]:
%sql
-- 皆様のユーザー名からスキーマ名を生成し、スキーマを作成
DROP TEMPORARY VARIABLE IF EXISTS database_name;
DECLARE database_name = concat("schema_", regexp_replace(session_user(), '[\.@-]', '_'));
CREATE DATABASE IF NOT EXISTS IDENTIFIER(database_name);
SELECT database_name;

In [0]:
# スキーマ名(データベース)
SCHEMA_NAME = _sqldf.first()["database_name"]
print(f"ハンズオンで使用するスキーマは {SCHEMA_NAME} です。")

## ボリュームの作成

カタログとスキーマ(データベース)の準備ができたので、ボリュームを作成します。

In [0]:
spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_NAME}.{VOLUME_NAME}")

以下のコマンドを実行して表示されるリンクをクリックしてボリュームを確認します。リンク先の画面は[カタログエクスプローラ](https://learn.microsoft.com/ja-jp/azure/databricks/catalog-explorer/)と呼ばれるものであり、Unity Catalogで管理されている資産をGUIから確認、操作することができます。

In [0]:
displayHTML(f"<a href='/explore/data/volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}' target='_blank'>作成したボリュームを表示</a>")

## データの取り込み

[requests](https://pypi.org/project/requests/)を用いてインターネットからCSVファイルを取得し、[dbutils.fs.put](https://learn.microsoft.com/ja-jp/azure/databricks/dev-tools/databricks-utils#dbutils-fs-cp)を用いてファイルをボリュームにコピーします。ここでは[ニューヨークの公開データ](https://health.data.ny.gov/)を使用します。

In [0]:
# データを取得して保存する
import requests

# CSVファイルを取得
response = requests.get('https://health.data.ny.gov/api/views/jxy9-yhdk/rows.csv')
csvfile = response.content.decode('utf-8')

# CSVファイルをボリュームに保存
dbutils.fs.put(f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}/babynames.csv", csvfile, True)

もう一度カタログエクスプローラにアクセスして、ボリュームにファイルが保存されていることを確認しましょう。

## データの確認

以下のコマンドは、SparkのPython APIである[PySpark](https://learn.microsoft.com/ja-jp/azure/databricks/pyspark/basics)を用いてCSVを読み込み内容を表示します。これが今回のハンズオンで使用する生データとなります。

In [0]:
# CSVファイルを読み込む
babynames = (
    spark.read.format("csv") # CSVフォーマット
    .option("header", "true") # ヘッダーあり
    .option("inferSchema", "true") # スキーマの推定
    .load(f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}/babynames.csv") # ファイルパスを指定
)
display(babynames)